# Institulac 

In [6]:
import re
import json
import time
from datetime import datetime

import pandas as pd
pd.set_option("max_rows",100)
#pd.set_option("display.max_columns",100)
pd.set_option("max_colwidth",1000)

import helium as h
from selenium.common.exceptions import NoSuchElementException

In [7]:
# DICT CAT-PRODS-TAB
with open('dict_tables.json') as file_json:
    dict_tables=json.loads(file_json.read())

In [8]:
# login =
# name_ins =
# usser =
# passw=

# login
browser = h.start_firefox('https://scienti.minciencias.gov.co/institulac2-war/')
sleep=0.8

#browser = h.start_firefox('https://scienti.minciencias.gov.co/institulac2-war/')
time.sleep(sleep)
h.click('Consulte Aquí')

time.sleep(sleep)
h.write('UNIVERSIDAD DE ANTIOQUIA',into='Digite el nombre de la Institución') # name ins

time.sleep(sleep)
h.click('Buscar')

time.sleep(sleep)
h.click(browser.find_element_by_id('list_instituciones'))

time.sleep(sleep)

time.sleep(sleep)
h.select('seleccione una','UNIVERSIDAD DE ANTIOQUIA') # name_ins

time.sleep(sleep)
h.write('annyarango',into='Usuario')                  # user

time.sleep(sleep)
h.write('1@Silver', into='Contraseña')                # passw

time.sleep(sleep)
h.click(h.Button('Ingresar'))

# cookie injection
time.sleep(sleep)
# implementation cookie injection

# get current cookie and store
new_cookie=browser.get_cookies()[0]
    
# create new_cookie with time_expire
time_expire = (datetime(2022,1,1) - datetime(1970,1,1)).total_seconds()
new_cookie['expiry'] = int(time_expire)
    
# delete cookie sites
browser.delete_all_cookies()
    
# add new cookie
browser.add_cookie(new_cookie)

# navigation 1
time.sleep(sleep)
h.click('Aval')

time.sleep(sleep)
h.click('Avalar grupos')

time.sleep(sleep)
h.click('Grupos Avalados')

# -- end login --

# list of total groups
#select max results per page
h.wait_until(h.Text('Ver Reporte').exists)
h.click(browser.find_element_by_xpath('//table[@id="grupos_avalados"]//select[@name="maxRows"]'))

time.sleep(sleep)
h.select(browser.find_element_by_xpath('//table[@id="grupos_avalados"]//select[@name="maxRows"]'),'100')

# catch 1: groups info [name, lider, cod,  link to producs]  
# schema
# empty df
# select max items per page
# while until end
# try:
    # catch table
    # preproces table
    # catch urls
    # add url colums
    # add df
    # click next page -> raise error
# except Nosuchelement:
    # break
    
# catch 1: list of groups
dfg=pd.DataFrame()
cont=True

while cont:
    
    try:
        # catch source
        time.sleep(sleep)
        source_g=browser.page_source
        
        # catch table
        time.sleep(sleep)
        df=pd.read_html(source_g, attrs={"id":"grupos_avalados"}, header=2)[0]
        
        # and preprocces it
        c=[x for x in df.columns if x.find('Unnamed:') == -1]
        dfgp=df[c][1:-1]
        print(dfgp.columns,dfgp.shape)
        
        # catch urls
        url=[a.get_attribute('href') for a in browser.find_elements_by_xpath('//table[@id="grupos_avalados"]//td[5]/a')]
        dfgp['Revisar'] = url
        dfg=dfg.append(dfgp)
        
        # click next page. this instruction rise error of the end. 
        h.click(browser.find_element_by_xpath('//table[@id="grupos_avalados"]//tr/td[3]/a'))
        
    except NoSuchElementException as e:
        
        print(e)
        print('out of cicle')
        break
        
    time.sleep(sleep)
    time.sleep(sleep)
    
dfg = dfg.reset_index(drop=True)
assert dfg.shape[0] == 324

Index(['Nombre del grupo', 'Nombre del líder', 'COL Grupo', 'Revisar'], dtype='object') (100, 4)
Index(['Nombre del grupo', 'Nombre del líder', 'COL Grupo', 'Revisar'], dtype='object') (100, 4)
Index(['Nombre del grupo', 'Nombre del líder', 'COL Grupo', 'Revisar'], dtype='object') (100, 4)
Index(['Nombre del grupo', 'Nombre del líder', 'COL Grupo', 'Revisar'], dtype='object') (24, 4)
Message: Unable to locate element: //table[@id="grupos_avalados"]//tr/td[3]/a

out of cicle


In [12]:
print(dfg['Nombre del grupo'][0:1].values[0],'\n',dfg['Revisar'][:1].values[0])

print(dfg['Nombre del grupo'][-1:].values[0],'\n',dfg['Revisar'][-1:].values[0])

Grupo Tandem en Nano-bio-física 
 https://scienti.minciencias.gov.co/institulac2-war/ReGrupoInstitucion/query.do?avalGr=T&codigoGrupo=&idGrupo=00000000020456
Promoción de la Salud 
 https://scienti.minciencias.gov.co/institulac2-war/ReGrupoInstitucion/query.do?avalGr=T&codigoGrupo=&idGrupo=00000000001767


In [15]:
dfg['Nombre del grupo'][15:16]

15    Grupo Académico de Epidemiología Clínica
Name: Nombre del grupo, dtype: object

In [16]:
time.sleep(sleep*2)

LD = [] # 
LP = []
LR = [] 
for idx in dfg.index[15:16]:
    
    # create db
    DBG = {}
    # part info group
    print(dfg.loc[idx,'Nombre del grupo'])

    # specific group url
    time.sleep(sleep)
    url_group = dfg.loc[idx,'Revisar']

    # go to url group
    time.sleep(sleep)
    browser.get(url_group)

    # catch two tables: info grupo and  members
    source=browser.page_source

    #info
    l_info=pd.read_html(source, match='Nombre Grupo')
    info_g=l_info[3].pivot(columns=0,values=1)
    
    # STORE INFO_GROUP
    DBG['Info_group'] = info_g

    # members
    l_int = pd.read_html(source,attrs={'id':'tblIntegrantes'},header=2)
    mem_g=l_int[0]
    
    # STORE_MEMBERS
    DBG['Members'] =  mem_g

    # Products

    #time.sleep(sleep*5) # time time time !!!
    h.wait_until(lambda: browser.find_element_by_xpath('//td[@id="bodyPrincipal"]//a[text()="Ver productos"]') is not None)
    h.click(browser.find_element_by_xpath('//td[@id="bodyPrincipal"]//a[text()="Ver productos"]'))

    # products by belongs to  # time time time
    #time.sleep(sleep*7)       # time time time
    h.wait_until(lambda: browser.find_element_by_xpath('//*[@id="ProdsPertenecia"]') is not None)
    h.click(browser.find_element_by_xpath('//*[@id="ProdsPertenecia"]'))

    time.sleep(sleep)
    url_products=browser.current_url


    # map all products, store those id categories that amount is different to 0 and id products asociated.
    # make queries with combinations of categories and products
    # make urls with diferent combinations of quieries
    # go to each of urls
    # load page source
    # catch table ( or tables) asociated with categories and products
    # store tables

    report = ''

    list_of_prods =[] #[[cat,prod],[cat,prod]...]

    # map all products and get products and subs diff to cero
    for i in browser.find_elements_by_xpath('//div[@id="accordionCatgP"]/h3'):

        report += i.text + '\n' 
        report += i.get_attribute('id') + '\n'     

        time.sleep(sleep)
        h.click(i)
        
        # cat
        cat_ = int(re.findall(r'\d+',i.text)[0])
        
        # create cat key in dict, for estore diferents products by this categori: 'NC_': {'ART_E':TABLE,
        #                                                                                 'ART_IMP':TABLE}
        if cat_ > 0:
            DBG[i.get_attribute('id')] = {}
            
        
        for j in browser.find_elements_by_xpath('//div[@aria-labelledby="%s"]/h3' % i.get_attribute('id')):

            report += '\t' + j.text + '\n' 
            report += '\t' + j.get_attribute('id') + '\n'
            
            #prod
            pro_ = int(re.findall(r'\d+', j.text)[0])

            if cat_ > 0 and pro_ > 0:  
                
                list_of_prods.append([i.get_attribute('id'),j.get_attribute('id')])

        time.sleep(sleep) 
        # h.click(a)
        h.click(i)

    # print(report)
    # print('\n')
    # print('--------------------------------')
    time.sleep(sleep*2)
    
    tables=[]
    
    for p in range(len(list_of_prods)):

            # make query 
            query='categoriaP=%s&subcategoriaP=%s&aval=P' % (list_of_prods[p][0],list_of_prods[p][1])

            # make url query
            url_query = url_products.split('?')[0] + '?' + query + '&' + url_products.split('?')[1]

            # retrieve id asociated tables
            table_id = dict_tables[list_of_prods[p][0]][list_of_prods[p][1]]

            # go to url product by group
            time.sleep(sleep)
        
            browser.get(url_query)

            # load page
            time.sleep(sleep)
            page_source = browser.page_source

            # catch tables
            if isinstance(table_id,str): # case one table

                # catch title table
                
                title_table = browser.find_element_by_xpath('//div/p[@class="titulo_tabla"]').text 
                # cathc table
                #print(table_id)
                time.sleep(sleep*2)
                table = pd.read_html(page_source,attrs={'id':table_id}, header=2)[0][1:-1]

                # store table
                DBG[list_of_prods[p][0]][list_of_prods[p][1]] = {table_id:table}
                # ---- in building ----

            elif isinstance(table_id, list): # case multiple tables

                for i in range(len(table_id)):

                    # catch title specific table 
                    title_table = browser.find_elements_by_xpath('//div/p[@class="titulo_tabla"]')[i].text

                    # catch table
                    table = pd.read_html(page_source,attrs={'id':table_id[i]}, header=2)[0][1:-1]

                    # store table
                    DBG[list_of_prods[p][0]][list_of_prods[p][1]] = {table_id[i]:table}
                    # -----------
    LD.append(DBG)
    LP.append(list_of_prods)
    LR.append(report)

Grupo Académico de Epidemiología Clínica


In [17]:
print(LR[0])

Nuevo Conocimiento (57)
NC_P
	Artículos publicados en revistas especializadas - Impresos (19)
	ART_IMP_P
	Artículos publicados en revistas especializadas - Electrónicos (32)
	ART_ELE_P
	Libros resultado de investigación (1)
	LIB_P
	Capítulos en Libro resultado de investigación (5)
	CAP_LIB_P
	Notas científicas (0)
	NOT_CIE_P
	Patentes (0)
	PAT_P
	Obras o productos de investigación creación en Artes, Arquitectura y Diseño (0)
	PRD_INV_ART_P
	Variedad Vegetal (0)
	VAR_VEG_P
	Nueva Raza Animal (0)
	VAR_ANI_P
	Poblaciones mejoradas de razas pecuarias (0)
	RAZ_PEC_P
	Traducciones Filológicas y Edición de Fuentes (0)
	TRA_FIL_P
Desarrollo Tecnológico e Innovación (0)
DTI_P
	Diseño industrial (0)
	DIS_IND_P
	Circuitos integrados (0)
	CIR_INT_P
	Software (0)
	SOFT_P
	Productos nutraceúticos (0)
	NUTRA_P
	Colecciones científicas (0)
	COL_CIENT_P
	Nuevos registros científicos (0)
	REG_CIENT_P
	Planta Piloto (0)
	PLT_PIL_P
	Prototipo Industrial (0)
	PRT_IND_P
	Secreto Industrial (0)
	SEC_IND_P
	P

In [75]:
LD[0]['FRH_P']

{'TES_DOC_P': {'TD_P_TABLE':   Unnamed: 0  \
  1          1   
  2          2   
  
                                                                                                                                                                                                                                                                                          Título  \
  1  Eficacia del Entrenamiento Aeróbico de Intervalos de Alta Intensidad (eAIAI) En Comparación con el Aeróbico Continuo (eAC) sobre la resistencia a la insulina, estructura y función del músculo esquelético en adultos con síndrome metabólico (SM): un ensayo clínico controlado aleatorio   
  2                                                                                                                                                                                     Efecto moderador de las funciones ejecutivas en la relación entre condiciones laborales y estrés laboral   
  
                            Autor  \


In [76]:
LP[0]

[['NC_P', 'ART_IMP_P'],
 ['NC_P', 'ART_ELE_P'],
 ['NC_P', 'LIB_P'],
 ['NC_P', 'CAP_LIB_P'],
 ['ASC_P', 'GEN_CONT_IMP_P'],
 ['ASC_P', 'EV_CIENT_P'],
 ['ASC_P', 'RED_P'],
 ['ASC_P', 'DOC_TRAB_P'],
 ['FRH_P', 'TES_DOC_P'],
 ['FRH_P', 'TES_MAST_P'],
 ['FRH_P', 'PROY_INV_DES_P'],
 ['FRH_P', 'ASE_CRE_CUR_P']]

In [77]:
assert len(LD)==len(LP)

In [78]:
def get_info(df):
    
    info= {
        'Nombre_Grupo' : df['Nombre Grupo'].dropna().iloc[0],

        'Nombre_Lider' : df['Nombre Líder'].dropna().iloc[0],

        'Nombre_Area'  : df['Nombre área'].dropna().iloc[0]
    }
    
    dfi = pd.DataFrame(info, index=[0])
  
    
    return dfi

In [103]:
DBEH = {
    
    'INFO_GROUP': 'TABLE',
    'MEMBERS':['Identificación Nacionalidad','Tiene afiliación con UdeA', 'Si no tiene afiliación UdeA diligencie el nombre de la Institución','Nro. Horas de dedicación semanales que avala el Coordinador de grupo'], # 2
       
    'NC_P': {'ART_IMP_P': {'ART_P_TABLE':['URL','DOI','Si no tiene URL o DOI agregue una evidencia en el repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
             'ART_ELE_P': {'ART_E_P_TABLE':['URL','DOI','Si no tiene URL o DOI agregue una evidencia en el repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
             'LIB_P':     {'LIB_P_TABLE':['Proyecto de investigación del cual se derivó el libro (Código-Título)','Financiador(es) del proyecto del cual se derivó el libro', 'Financiador(es) de la publicación','Autores','Citas recibidas (si tiene)','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
             'CAP_LIB_P': {'CAP_LIB_P_TABLE':['Proyecto de investigación del cual se derivó el libro que contiene el capítulo (Código-Título)','Financiador del proyecto del cual se derivó el libro que contiene el capítulo','Financiador de la publicación','Autores','Citas recibidas (si tiene)','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
             'NOT_CIE_P': {'NOT_CIE_P_TABLE':['URL','DOI','Si no tiene URL o DOI genere una evidencia en el repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
             'PAT_P':     {'PAT_P_TABLE':['Autores', 'Examen de fondo favorable','Examen preliminar internacional favorable','Adjunta opiniones escritas de la bUsqueda internacional','Contrato de explotación','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']}, #  123 -1
             'PRD_INV_ART_P': {'PAAD_P_TABLE':['Autores','Tiene certificado institucional de la obra','Tiene certificado de la entidad que convoca al evento en el que participa','Tiene certificado de la entidad que convoca al premio en el que obtiene','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']}, # 1 2 3 -1
             'VAR_VEG_P':     {'VV_P_TABLE':['Autores','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
             'VAR_ANI_P':     {'VA_P_TABLE':['Autores','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
             'RAZ_PEC_P':     {'RAZ_PEC_P_TABLE':['Autores','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
             'TRA_FIL_P': {'TRA_FIL_P_TABLE':['Proyecto de investigación del cual se derivó el libro (Código-Título)','Financiador(es) del proyecto del cual se derivó el libro','Financiador(es) de la publicación','Autores','Citas recibidas (si tiene)','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']}
            },
     'DTI_P': {'DIS_IND_P': {'DI_P_TABLE':['Autores','Contrato (si aplica)','Nombre comercial (si aplica)','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'CIR_INT_P': {'ECI_P_TABLE':['Autores','Contrato (si aplica)','Nombre comercial (si aplica)','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'SOFT_P': {'SF_P_TABLE':['Autores','Contrato (si aplica)','Nombre comercial (si aplica)','TRL','Agregue la evidencia verificada al repositorio digital y copie el link del archivo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'NUTRA_P': {'NUTRA_P_TABLE':['Autores','Agregue la evidencia verificada al repositorio digital y copie el link del archivo en este campo','¿El producto cumple con los requisitos para ser avalado?']}, # add
              'COL_CIENT_P': {'COL_CIENT_P_TABLE':['Autores','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo', '¿El producto cumple con los requisitos para ser avalado?']},
              'REG_CIENT_P': {'REG_CIENT_P_TABLE':['Autores','Contrato licenciamiento (si aplica)','Agregue las evidencias verificadas al repositorio digital y copie el link del archivo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'PLT_PIL_P': {'PP_P_TABLE':['Autores','Agregue la evidencia verificada al repositorio digital y copie el link del archivo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'PRT_IND_P': {'PI_P_TABLE':['Autores','Nombre comercial (si aplica)','TRL','Agregue la evidencia verificada al repositorio digital y copie el link del archivo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'SEC_IND_P': {'SE_P_TABLE':['Autores','Agregue la evidencia verificada al repositorio digital y copie el link del archivo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'PROT_VIG_EPID_P': {'PROT_VIG_EPID_P_TABLE':['Autores','Agregue la evidencia verificada al repositorio digital y copie el link del archivo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'EMP_BSE_TEC_P': {'EBT_P_TABLE':['Autores','Agregue la evidencia verificada al repositorio digital y copie el link del archivo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'EMP_CRE_CUL_P': {'ICC_P_TABLE':['Autores','Agregue la evidencia verificada al repositorio digital y copie el link del archivo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'INN_GES_EMP_P': {'IG_P_TABLE':['Autores','Contrato (si aplica)','Nombre comercial (si aplica)','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'INN_PROC_P': {'IPP_P_TABLE':['Autores','Contrato (si aplica)','Nombre comercial (si aplica)','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'REG_NORM_REGL_LEG_P': {'RNR_P_TABLE':['Autores','Contrato (si aplica)','Convenio (si aplica)','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'CONP_TEC_P': {'CONP_TEC_P_TABLE':['Autores','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'REG_AAD_P': {'AAAD_P_TABLE':['Autores','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'SIG_DIS_P': {'SD_P_TABLE':['Autores','Contrato licenciamiento (si aplica)','Agregue las evidencias verificadas al repositorio digital y copie el link del archivo en este campo','¿El producto cumple con los requisitos para ser avalado?']}
              },
    'ASC_P': {'GEN_CONT_IMP_P': {'GC_I_P_TABLE_5':['Autores','Citas recibidas (si tiene)','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'PASC_P': {'PASC_FOR_P_TABLE':['Proyecto/Código','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?'],
               'PASC_TRA_P_TABLE':['Proyecto/Código','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?'],
               'PASC_GEN_P_TABLE':['Proyecto/Código','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?'],
               'PASC_CAD_P_TABLE':['Proyecto/Código','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'DC_P': {'DC_CD_P_TABLE':['Proyecto/Código','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?'],
               'DC_CON_P_TABLE':['Medio de verificación','Proyecto/Código','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?'],
               'DC_TRA_P_TABLE':['Medio de verificación','Proyecto/Código','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?'],
               'DC_DES_P_TABLE':['Medio de verificación','Proyecto/Código','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']}},
    
    'FRH_P': {'TES_DOC_P': {'TD_P_TABLE':['Número de cédula del graduado','¿La fecha fin coincide con la fecha de grado del estudiante?','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},  # 1 -1
              'TES_MAST_P': {'TM_P_TABLE':['Número de cédula del graduado','¿La fecha fin coincide con la fecha de grado del estudiante?','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']}, # 1 -1
              'TES_PREG_P': {'TP_P_TABLE':['Número de cédula del graduado','¿La fecha fin coincide con la fecha de grado del estudiante?','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']}, # 1 -1
              'ASE_PRG_ACA_P': {'APGA_P_TABLE':['Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'ASE_CRE_CUR_P': {'ACC_P_TABLE':['Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'ASE_PRG_ONDAS_P': {'APO_P_TABLE':['Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']}}
}

In [104]:
d = {
                '1': 'C',
                '2': 'D',
                '3': 'E',
                '4': 'F',
                '5': 'G',
                '6': 'H',
                '7': 'I',
                '8': 'J',
                '9': 'K',
                '10': 'L',
                '11': 'M',
                '12': 'N',
                '13': 'O',
                '14': 'P',
                '15': 'Q'
}

In [105]:
def clean_df(df):
    'remove innecesari collums'
    c=[x for x in df.columns if x.find('Unnamed:') == -1 and  x.find('Revisar') == -1]
    dfc=df[c]
    return dfc

In [106]:
def format_df(df, sheet_name, start_row, writer,eh, veh =None):
    'format headers'
    
    df.to_excel(writer,sheet_name, startrow = start_row+1, startcol=2,index = False)

    # Get the xlsxwriter workbook and worksheet objects.
    worksheet = writer.sheets[sheet_name]
    
    #form merge cells
    start,end = 1,df.shape[1]

    m_range = d.get(str(start)) + str(start_row + 1) + ':' + d.get(str(end)) + str(start_row +1)

    worksheet.merge_range(m_range, 'Información suministrada por la Vicerrectoría de Investigación', merge_format)
    
    # for merge cells
    _m_range = d.get(str(end+1)) + str(start_row +1) + ':' +  d.get(str(end+len(eh))) + str(start_row +1)
    
    worksheet.merge_range(_m_range, 'Validación del Centro, Instituto o Corporación', merge_format)
        
    worksheet.set_row_pixels(start_row+1, 120)
    #worksheet.set_column('C:C',30,general)
    
    # SET COLUMS BY SHEET
    
  '''  if:
        worksheet.set_column(
        '''
    if sheet_name == '3.Integrantes grupo':
        
        worksheet.set_column('C:C', 10,general)
        worksheet.set_column('D:D', 30,general)
        worksheet.set_column('E:G', 15,general)
        
    elif sheet_name== 'NC_P':
        
        worksheet.set_column('C:C', 30,general)
        worksheet.set_column('D:K', 20,general)
    elif sheet_name == 'FRH_P':
        
        worksheet.set_column('C:C', 30,general)
        worksheet.set_column('D:K', 20,general)
    else:
        worksheet.set_column('C:C', 30,general)
        worksheet.set_column('D:K', 20,general)
    
    worksheet.set_column('A:A', 15)
    worksheet.set_column('B:B', 5)
        
    worksheet.write(start_row+1, 0, 'VoBo de VRI', merge_format)
    # Add a header format.
    
    fmt_header = workbook.add_format({
        'bold': True,
        'align': 'center',    
        'text_wrap': True,
        'valign': 'vcenter',
        'fg_color': '#33A584',
        'font_color': '#FFFFFF',
        'border': 1})
    
    
    # Write the column headers with the defined format.
    for col_num, value in enumerate(df.columns.values):
        worksheet.write(start_row+1, col_num + 2, value, fmt_header)
        
    # write extra headers
    for col_num, value in enumerate(eh):
        worksheet.write(start_row+1, col_num + df.shape[1] + 2, value, fmt_header)

In [107]:
def format_info(df, writer, sheet_name):
    
    '''format worksheet'''
    
    workbook=writer.book
    
    normal=workbook.add_format({'font_size':12,'text_wrap':True})
    
    merge_format = workbook.add_format({
    'bold': 1,
    'border':1,
    'text_wrap': True,    
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black'})
    
    fmt_header = workbook.add_format({
        'align': 'center',    
        'text_wrap': True,
        'valign': 'top',
        'fg_color': '#33A584',
        'font_color': '#FFFFFF',
        'border': 1})
    
    # write df
    start_row = 6
    start_col = 3
    
    df.to_excel(writer, sheet_name, startrow =start_row, startcol=start_col,index = False)

    # get worksheet object
    worksheet = writer.sheets[sheet_name]
    
    for col_num, value in enumerate(df.columns.values):
        worksheet.write(start_row, col_num + 3, value, fmt_header)
    
    #Prepare image insertion: See → https://xlsxwriter.readthedocs.io/example_images.html
    worksheet.set_column('A:A', 15)
    worksheet.set_column('B:B', 15)
    worksheet.insert_image('A1', 'img/udea.jpeg')
    
    # title 1 UNIVERSIDAD DE ANTIOQUIA
    title = workbook.add_format({'font_size':16,'center_across':True})

    # title 2 Vicerrectoria de Investigación
    title2 = workbook.add_format({'font_size':16,'center_across':True})
   
    # sub title 2 datos identificacion contacto
    title3 = workbook.add_format({'font_size':12,'center_across':True})
    
    # merge d1:f1
    worksheet.merge_range('D1:F1', 'UNIVERSIDAD DE ANTIOQUIA', title)
        
    # merge d2:f2
    worksheet.merge_range('D2:F2', ' Vicerrectoria de Investigación', title2)
    
    # merge d3:f3
    worksheet.merge_range('D3:F3', ' Datos de identificación y contacto', title3)
    
    # D5: F5
    worksheet.merge_range('D5:E5','Número inscripcion a la convocatoria:',merge_format)
    worksheet.write('F5','#',merge_format)
    
    # d6:f6
    worksheet.merge_range('D6:F6','Identificación del Grupo',merge_format)
        
    # d9:f9
    worksheet.merge_range('D10:F10','Identificación del Centro de Investigación',merge_format)
    # write 
    a='Nombre del Centro, Instituto o Corporación'
    worksheet.write('D11',a, fmt_header)
    worksheet.set_column('D11:D11',30, fmt_header)
    
    b='Nombre completo del Jefe de Centro, Instituto o Corporación'
    worksheet.write('E11',b, fmt_header) 
    worksheet.set_column('E11:E11',30, fmt_header)
    
    c='Email'
    worksheet.write('F11',c, fmt_header) 
    worksheet.set_column('F11:F11',30, fmt_header)
    
    # d13:f13
    worksheet.merge_range('D13:F13','Identificación de quien diligencia el formato',merge_format)
    a='Nombre completo del encargado de diligenciar el formato'
    worksheet.write('D14',a, fmt_header)
    worksheet.set_column('D14:D14',30, normal)
    
    b='Email'
    worksheet.write('E14',b, fmt_header) 
    worksheet.set_column('E14:E14',30, normal)
    
    c='Teléfono de contacto'
    worksheet.write('F14',c, fmt_header) 
    worksheet.set_column('F14:F14',30, normal)

In [108]:
'''def format_df(df, sheet_name, start_row, writer):
    'format headers'
    
    # Get the xlsxwriter workbook and worksheet objects.
    worksheet = writer.sheets[sheet_name]

    # Add a header format.
    
    fmt_header = workbook.add_format({
        'bold': True,
        'align': 'center',    
        'text_wrap': True,
        'valign': 'top',
        'fg_color': '#33A584',
        'font_color': '#FFFFFF',
        'border': 1})
    
    
    # Write the column headers with the defined format.
    for col_num, value in enumerate(df.columns.values):
        worksheet.write(start_row, col_num + 1, value, fmt_header) # !!!'''

"def format_df(df, sheet_name, start_row, writer):\n    'format headers'\n    \n    # Get the xlsxwriter workbook and worksheet objects.\n    worksheet = writer.sheets[sheet_name]\n\n    # Add a header format.\n    \n    fmt_header = workbook.add_format({\n        'bold': True,\n        'align': 'center',    \n        'text_wrap': True,\n        'valign': 'top',\n        'fg_color': '#33A584',\n        'font_color': '#FFFFFF',\n        'border': 1})\n    \n    \n    # Write the column headers with the defined format.\n    for col_num, value in enumerate(df.columns.values):\n        worksheet.write(start_row, col_num + 1, value, fmt_header) # !!!"

In [109]:
# https://stackoverflow.com/questions/42370977/how-to-save-a-new-sheet-in-an-existing-excel-file-using-pandas

In [110]:
# store info group in dict
    # each key store list of tables related with

In [111]:
# ONE GROUP IMPLEMENTATION
idxx = 34

# DATA
DBG = LD[0]

### excel name
name = 'AA_Plantilla_Formato de verificación de información_GrupLAC_894-2021_%s'

# initialize object= output excel file
writer = pd.ExcelWriter(name % dfg.loc[idxx,'COL Grupo']+'.xlsx', engine='xlsxwriter')

#Global variables
abstract_text='VERIFICACIÓN DE INFORMACIÓN PARA OTORGAR AVAL A LOS GRUPOS DE INVESTIGACIÓN  E INVESTIGADORES PARA SU PARTICIPACIÓN EN LA CONVOCATORIA 894 DE 2021 DE MINCIENCIAS'
instructions='''Los grupos de investigación e investigadores de la Universidad de Antioquia que deseen participar en la Convocatoria Nacional para el reconocimiento y medición de grupos de investigación, desarrollo tecnológico o de innovación y para el reconocimiento de investigadores del Sistema Nacional de Ciencia, Tecnología e Innovación - SNCTI, 894 de 2021, deben presentar la información actualizada en las plataformas CvLAC y GrupLAC validada por el Centro de Investigación en el presente formato, y respaldada en el repositorio digital de evidencias dispuesto para este fin, para la obtención del aval institucional por parte de la Vicerrectoría de Investigación. 

La información a validar corresponde a los años 2019-2020 y aquella que entra en la ventana de observación y debe ser modificada según el Modelo de medición de grupos. La validación comprende:

1. Verificación de la vinculación de los integrantes a la Universidad de Antioquia y al grupo de investigación.  Diligenciar los campos solicitados. 

2. Verificación de la producción de GNC, DTeI, ASC y FRH, en los campos habilitados en cada hoja de este formato. Las evidencias requeridas para los productos deben ser anexadas al repositorio digital asignado al grupo y se deben enlazar a cada producto.  

Este documento debe ser diligenciado en línea.

De antemano, la Vicerrectoría de Investigación agradece su participación en este ejercicio, que resulta de vital importancia para llevar a buen término la Convocatoria de Reconocimiento y Medición de Grupos de Investigación
'''
#Final part of the first sheet
datos=clean_df(pd.read_excel('https://github.com/restrepo/InstituLAC/raw/main/data/template_data.xlsx'))

#Capture xlsxwriter object 
# IMPORTANT → workbook is the same object used in the official document at https://xlsxwriter.readthedocs.io
workbook=writer.book
#***************
#Styles as explained in https://xlsxwriter.readthedocs.io
general=workbook.add_format({'text_wrap':True})
title=workbook.add_format({'font_size':28,'center_across':True})
subtitle=workbook.add_format({'font_size':24,'center_across':True})
abstract=workbook.add_format({'font_size':20,'center_across':True,'text_wrap':True})
normal=workbook.add_format({'font_size':12,'text_wrap':True})

merge_format = workbook.add_format({
    'bold': 1,
    'border':1,
    'text_wrap': True,    
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'blue'})

fmt_header = workbook.add_format({
    'bold': True,
    'align': 'center',    
    'text_wrap': True,
    'valign': 'top',
    'fg_color': '#33A584',
    'font_color': '#FFFFFF',
    'border': 1})
#***************
#Creates the first work-sheet
#IMPORTANT → worksheet is the same object  used in the official document at https://xlsxwriter.readthedocs.io
worksheet=workbook.add_worksheet("1.Presentación")
#Prepare image insertion: See → https://xlsxwriter.readthedocs.io/example_images.html
worksheet.set_column('A:A', 15)
worksheet.set_column('B:B', 15)
worksheet.insert_image('A1', 'img/udea.jpeg')
#Prepare text insertion: See  → https://xlsxwriter.readthedocs.io/example_images.html
worksheet.set_column('C:C', 140,general)
worksheet.set_row_pixels(0, 60)
#Texts
worksheet.write('C1', 'UNIVERSIDAD DE ANTIOQUIA',title)
worksheet.set_row_pixels(2, 60)
worksheet.write('C3', 'VICERRECTORÍA DE INVESTIGACIÓN',subtitle)
worksheet.set_row_pixels(5, 100)
worksheet.write('C6', abstract_text,abstract)
worksheet.set_row_pixels(8, 40)
worksheet.write('C9','PRESENTACIÓN DEL EJERCICIO',
                workbook.add_format({'font_size':18,'center_across':True}) )
worksheet.set_row_pixels(10, 320)
worksheet.write('C11',instructions,normal )
#*** ADD PANDAS DATAFRAME IN SPECIFIC POSITION ****
#Add a data Frame in some specific position. See → https://stackoverflow.com/a/43510881/2268280
#                                       See also → https://xlsxwriter.readthedocs.io/working_with_pandas.html
writer.sheets["1.Presentación"]=worksheet
datos.to_excel(writer,sheet_name="1.Presentación",startrow=12,startcol=2,index=False)
#**************************************************
#Fix columns heights for long text
worksheet.set_row_pixels(17, 40)
worksheet.set_row_pixels(18, 40)
worksheet.set_row_pixels(19, 40)
worksheet.set_row_pixels(20, 40)
worksheet.set_row_pixels(22, 40)

In [112]:
''' # Add extra headers
worksheet2.write(1, 0, 'VoBo de VRI', merge_format)
#New columns
extra_url='Si no tiene URL o DOI agregue una evidencia en el repositorio digital y genere un hipervínculo en este campo'
cols=['DOI',extra_url,'¿El producto cumple con los requisitos para ser avalado?']
for col , value in enumerate(cols):
    worksheet2.write(1, col+2+table.columns.size, value, fmt_header)
worksheet2.set_column('L:L',20)
worksheet2.set_column('M:M',20)
#Creates a set of cells with a drop-down menu Sí/No. See → https://xlsxwriter.readthedocs.io/working_with_data_validation.html
worksheet2.data_validation('M3:M{}'.format(table.shape[0]+2), {'validate': 'list',
                                  'source': ['Sí', 'No']})

workbook.close()
'''

" # Add extra headers\nworksheet2.write(1, 0, 'VoBo de VRI', merge_format)\n#New columns\nextra_url='Si no tiene URL o DOI agregue una evidencia en el repositorio digital y genere un hipervínculo en este campo'\ncols=['DOI',extra_url,'¿El producto cumple con los requisitos para ser avalado?']\nfor col , value in enumerate(cols):\n    worksheet2.write(1, col+2+table.columns.size, value, fmt_header)\nworksheet2.set_column('L:L',20)\nworksheet2.set_column('M:M',20)\n#Creates a set of cells with a drop-down menu Sí/No. See → https://xlsxwriter.readthedocs.io/working_with_data_validation.html\nworksheet2.data_validation('M3:M{}'.format(table.shape[0]+2), {'validate': 'list',\n                                  'source': ['Sí', 'No']})\n\nworkbook.close()\n"

In [113]:
DBEH.keys()

dict_keys(['INFO_GROUP', 'MEMBERS', 'NC_P', 'DTI_P', 'ASC_P', 'FRH_P'])

In [114]:
# INFO GROUP
df=get_info(DBG['Info_group'])
format_info(df, writer, '2.Datos de contacto')

# WORKSHEET 1
df = clean_df(DBG['Members']) 
eh = DBEH['MEMBERS']
format_df(df, '3.Integrantes grupo', 1, writer, eh, veh=0) #### veh = 0

In [115]:
### NC_P ### WORKSHEET 2
     
var_nc = 0

try:
    df=clean_df(DBG['NC_P']['ART_IMP_P']['ART_P_TABLE'])

    #df.to_excel(writer,sheet_name='NC_P',startrow = var_nc+1)
    
    eh=DBEH['NC_P']['ART_IMP_P']['ART_P_TABLE']

    format_df(df, 'NC_P', var_nc, writer,eh)
        
    var_nc += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['NC_P']['ART_ELE_P']['ART_E_P_TABLE'])
    
    #df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    eh=DBEH['NC_P']['ART_ELE_P']['ART_E_P_TABLE']

    format_df(df, 'NC_P', var_nc, writer,eh)
        
    var_nc += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['NC_P']['LIB_P']['LIB_P_TABLE'])

    #df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    eh=DBEH['NC_P']['LIB_P']['LIB_P_TABLE']

    format_df(df, 'NC_P', var_nc, writer,eh)
        
    var_nc += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['NC_P']['CAP_LIB_P']['CAP_LIB_P_TABLE'])

    #df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    eh=DBEH['NC_P']['CAP_LIB_P']['CAP_LIB_P_TABLE']

    format_df(df, 'NC_P', var_nc, writer,eh)
        
    var_nc += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['NC_P']['NOT_CIE_P']['NOT_CIE_P_TABLE'])
    
    #df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    eh=DBEH['NC_P']['NOT_CIE_P']['NOT_CIE_P_TABLE']

    format_df(df, 'NC_P', var_nc, writer,eh)
        
    var_nc += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['NC_P']['PAT_P']['PAT_P_TABLE']) ###### veh=1
    
    #df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    eh=DBEH['NC_P']['PAT_P']['PAT_P_TABLE']

    format_df(df, 'NC_P', var_nc, writer,eh, veh=1)
        
    var_nc += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['NC_P']['PRD_INV_ART_P']['PAAD_P_TABLE']) ###### veh = 1

    #df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    eh=DBEH['NC_P']['PRD_INV_ART_P']['PAAD_P_TABLE']

    format_df(df, 'NC_P', var_nc, writer,eh, veh=1)
        
    var_nc += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['NC_P']['VAR_VEG_P']['VV_P_TABLE'])

    #df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    eh=DBEH['NC_P']['VAR_VEG_P']['VV_P_TABLE']

    format_df(df, 'NC_P', var_nc, writer,eh)
        
    var_nc += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['NC_P']['VAR_ANI_P']['VA_P_TABLE'])

    #df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    eh=DBEH['NC_P']['VAR_ANI_P']['VA_P_TABLE']

    format_df(df, 'NC_P', var_nc, writer,eh)
        
    var_nc += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['NC_P']['RAZ_PEC_P']['RAZ_PEC_P_TABLE'])
    
    #df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    eh=DBEH['NC_P']['RAZ_PEC_P']['RAZ_PEC_P_TABLE']

    format_df(df, 'NC_P', var_nc, writer,eh)
        
    var_nc += df.shape[0] + 3

except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['NC_P']['TRA_FIL_P']['TRA_FIL_P_TABLE'])
    
    #df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    eh=DBEH['NC_P']['TRA_FIL_P']['TRA_FIL_P_TABLE']

    format_df(df, 'NC_P', var_nc, writer,eh)
        
    var_nc += df.shape[0] + 3
    
except KeyError as e:
    
    pass

#### DTI_P

var_dt = 0
try:

    df=clean_df(DBG['DTI_P']['DIS_IND_P']['DI_P_TABLE'])
    
    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['DIS_IND_P']['DI_P_TABLE']

    format_df(df, 'DTI_P', var_dt, writer,eh)
        
    var_dt += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['DTI_P']['CIR_INT_P']['ECI_P_TABLE'])
    
    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['CIR_INT_P']['ECI_P_TABLE']

    format_df(df, 'DTI_P', var_dt, writer,eh)
        
    var_dt += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['DTI_P']['SOFT_P']['SF_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['SOFT_P']['SF_P_TABLE']

    format_df(df, 'DTI_P', var_dt, writer,eh)
        
    var_dt += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['DTI_P']['NUTRA_P']['NUTRA_P_TABLE'])
    
    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_nc)
    
    eh=DBEH['DTI_P']['NUTRA_P']['NUTRA_P_TABLE']

    format_df(df, 'DTI_P', var_dt, writer,eh)
        
    var_dt += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['DTI_P']['COL_CIENT_P']['COL_CIENT_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['COL_CIENT_P']['COL_CIENT_P_TABLE']

    format_df(df, 'DTI_P', var_dt, writer,eh)
        
    var_dt += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['DTI_P']['REG_CIENT_P']['REG_CIENT_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['REG_CIENT_P']['REG_CIENT_P_TABLE']

    format_df(df, 'DTI_P', var_dt, writer,eh)
        
    var_dt += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['DTI_P']['PLT_PIL_P']['PP_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['PLT_PIL_P']['PP_P_TABLE']

    format_df(df, 'DTI_P', var_dt, writer,eh)
        
    var_dt += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['DTI_P']['PRT_IND_P']['PI_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['PRT_IND_P']['PI_P_TABLE']

    format_df(df, 'DTI_P', var_dt, writer,eh)
        
    var_dt += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['DTI_P']['SEC_IND_P']['SE_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['SEC_IND_P']['SE_P_TABLE']

    format_df(df, 'DTI_P', var_dt, writer,eh)
        
    var_dt += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['DTI_P']['PROT_VIG_EPID_P']['PROT_VIG_EPID_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['PROT_VIG_EPID_P']['PROT_VIG_EPID_P_TABLE']

    format_df(df, 'DTI_P', var_dt, writer,eh)
        
    var_dt += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['DTI_P']['EMP_BSE_TEC_P']['EBT_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['EMP_BSE_TEC_P']['EBT_P_TABLE']

    format_df(df, 'DTI_P', var_dt, writer,eh)
        
    var_dt += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['DTI_P']['EMP_CRE_CUL_P']['ICC_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['EMP_CRE_CUL_P']['ICC_P_TABLE']

    format_df(df, 'DTI_P', var_dt, writer,eh)
        
    var_dt += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['DTI_P']['INN_GES_EMP_P']['IG_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['INN_GES_EMP_P']['IG_P_TABLE']

    format_df(df, 'DTI_P', var_dt, writer,eh)
        
    var_dt += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['DTI_P']['INN_PROC_P']['IPP_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['INN_PROC_P']['IPP_P_TABLE']

    format_df(df, 'DTI_P', var_dt, writer,eh)
        
    var_dt += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['DTI_P']['REG_NORM_REGL_LEG_P']['RNR_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['REG_NORM_REGL_LEG_P']['RNR_P_TABLE']

    format_df(df, 'DTI_P', var_dt, writer,eh)
        
    var_dt += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['DTI_P']['CONP_TEC_P']['CONP_TEC_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['CONP_TEC_P']['CONP_TEC_P_TABLE']

    format_df(df, 'DTI_P', var_dt, writer,eh)
        
    var_dt += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['DTI_P']['REG_AAD_P']['AAAD_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['REG_AAD_P']['AAAD_P_TABLE']

    format_df(df, 'DTI_P', var_dt, writer,eh)
        
    var_dt += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['DTI_P']['SIG_DIS_P']['SD_P_TABLE'])
    
    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['SIG_DIS_P']['SD_P_TABLE']

    format_df(df, 'DTI_P', var_dt, writer,eh)
        
    var_dt += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

######  ASC

var_as = 0

try:
    df=clean_df(DBG['ASC_P']['GEN_CONT_IMP_P']['GC_I_P_TABLE_5'])

    #df.to_excel(writer,sheet_name='ASC_P',startrow = var_as)
    
    eh=DBEH['ASC_P']['GEN_CONT_IMP_P']['GC_I_P_TABLE_5']

    format_df(df, 'ASC_P', var_as, writer,eh)
        
    var_as += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass                  

try:
    df=clean_df(DBG['ASC_P']['PASC_P']['PASC_FOR_P_TABLE'])
    
    #df.to_excel(writer,sheet_name='ASC_P',startrow = var_as)
    
    eh=DBEH['ASC_P']['GEN_CONT_IMP_P']['GC_I_P_TABLE_5']

    format_df(df, 'ASC_P', var_as, writer,eh)
        
    var_as += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['ASC_P']['PASC_P']['PASC_TRA_P_TABLE'])

    #df.to_excel(writer,sheet_name='ASC_P',startrow = var_as)
    
    eh=DBEH['ASC_P']['PASC_P']['PASC_TRA_P_TABLE']

    format_df(df, 'ASC_P', var_as, writer,eh)
        
    var_as += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['ASC_P']['PASC_P']['PASC_GEN_P_TABLE'])

    #df.to_excel(writer,sheet_name='ASC_P',startrow = var_as)
    
    eh=DBEH['ASC_P']['PASC_P']['PASC_GEN_P_TABLE']

    format_df(df, 'ASC_P', var_as, writer,eh)
        
    var_as += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['ASC_P']['PASC_P']['PASC_CAD_P_TABLE'])

    #df.to_excel(writer,sheet_name='ASC_P',startrow = var_as)
    
    eh=DBEH['ASC_P']['PASC_P']['PASC_CAD_P_TABLE']

    format_df(df, 'ASC_P', var_as, writer,eh)
        
    var_as += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['ASC_P']['DC_P']['DC_CD_P_TABLE'])

    #df.to_excel(writer,sheet_name='ASC_P',startrow = var_as)
    
    eh=DBEH['ASC_P']['DC_P']['DC_CD_P_TABLE']

    format_df(df, 'ASC_P', var_as, writer,eh)
        
    var_as += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['ASC_P']['DC_P']['DC_CON_P_TABLE'])

    #df.to_excel(writer,sheet_name='ASC_P',startrow = var_as)
    
    eh=DBEH['ASC_P']['DC_P']['DC_CON_P_TABLE']

    format_df(df, 'ASC_P', var_as, writer,eh)
        
    var_as += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['ASC_P']['DC_P']['DC_TRA_P_TABLE'])

    #df.to_excel(writer,sheet_name='ASC_P',startrow = var_as)
    
    eh=DBEH['ASC_P']['DC_P']['DC_TRA_P_TABLE']

    format_df(df, 'ASC_P', var_as, writer,eh)
        
    var_as += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['ASC_P']['DC_P']['DC_TRA_P_TABLE'])

    #df.to_excel(writer,sheet_name='ASC_P',startrow = var_as)
    
    eh=DBEH['ASC_P']['DC_P']['DC_TRA_P_TABLE']

    format_df(df, 'ASC_P', var_as, writer,eh)
        
    var_as += df.shape[0] + 3
    
except KeyError as e:
    
    pass

# FRH

var_rh = 0

try:
    df=clean_df(DBG['FRH_P']['TES_DOC_P']['TD_P_TABLE'])  ### ,veh = 2

    #df.to_excel(writer,sheet_name='FRH_P',startrow = var_rh)
    
    eh=DBEH['FRH_P']['TES_DOC_P']['TD_P_TABLE']

    format_df(df, 'FRH_P', var_rh, writer, eh)
        
    var_rh += df.shape[0] + 3

except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['FRH_P']['TES_MAST_P']['TM_P_TABLE']) ### veh = 2

    #df.to_excel(writer,sheet_name='FRH_P',startrow = var_rh)
    
    eh=DBEH['FRH_P']['TES_MAST_P']['TM_P_TABLE']

    format_df(df, 'FRH_P', var_rh, writer,eh)
        
    var_rh += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['FRH_P']['TES_PREG_P']['TP_P_TABLE']) ### veh = 2

    #df.to_excel(writer,sheet_name='FRH_P',startrow = var_rh)
    
    eh=DBEH['FRH_P']['TES_PREG_P']['TP_P_TABLE']

    format_df(df, 'FRH_P', var_rh, writer,eh,veh = 2)
        
    var_rh += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['FRH_P']['ASE_PRG_ACA_P']['APGA_P_TABLE']) 

    #df.to_excel(writer,sheet_name='FRH_P',startrow = var_rh)
    
    eh=DBEH['FRH_P']['ASE_PRG_ACA_P']['APGA_P_TABLE']

    format_df(df, 'FRH_P', var_rh, writer,eh)
        
    var_rh += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['FRH_P']['ASE_CRE_CUR_P']['ACC_P_TABLE'])

    #df.to_excel(writer,sheet_name='FRH_P',startrow = var_rh)
    
    eh=DBEH['FRH_P']['ASE_CRE_CUR_P']['ACC_P_TABLE']

    format_df(df, 'FRH_P', var_rh, writer,eh)
        
    var_rh += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['FRH_P']['ASE_PRG_ONDAS_P']['APO_P_TABLE'])

    #df.to_excel(writer,sheet_name='FRH_P',startrow = var_rh)
    
    eh=DBEH['FRH_P']['ASE_PRG_ONDAS_P']['APO_P_TABLE']

    format_df(df, 'FRH_P', var_rh, writer,eh)
        
    var_rh += df.shape[0] + 3
    
except KeyError as e:
    
    pass

writer.save()

# ------

In [37]:
'''
schema for store tables

DBG = {
    
    'INFO_GROUP': 'TABLE',
    'MEMBERS':'TABLE',
       
    'NC_P': {'ART_IMP_P': {'ART_P_TABLE':'TABLE'},
             'ART_ELE_P': {'ART_E_P_TABLE':'TABLE'},
             'LIB_P':     {'LIB_P_TABLE':'TABLE'},
             'CAP_LIB_P': {'CAP_LIB_P_TABLE':'TABLE'},
             'NOT_CIE_P': {'NOT_CIE_P_TABLE':'TABLE'},
             'PAT_P':     {'PAT_P_TABLE':'TABLE'},
             'PRD_INV_ART_P': {'PAAD_P_TABLE':'TABLE'},
             'VAR_VEG_P':     {'VV_P_TABLE':'TABLE'},
             'VAR_ANI_P':     {'VA_P_TABLE':'TABLE'},
             'RAZ_PEC_P':     {'RAZ_PEC_P_TABLE':'TABLE'},
             'TRA_FIL_P': {'TRA_FIL_P_TABLE':'TABLE'}
            },
     'DTI_P': {'DIS_IND_P': {'DI_P_TABLE':'TABLE'},
              'CIR_INT_P': {'ECI_P_TABLE':'TABLE'},
              'SOFT_P': {'SF_P_TABLE':'TABLE'},
              'NUTRA_P': {'NUTRA_P_TABLE':'TABLE'},
              'COL_CIENT_P': {'COL_CIENT_P_TABLE':'TABLE'},
              'REG_CIENT_P': {'REG_CIENT_P_TABLE':'TABLE'},
              'PLT_PIL_P': {'PP_P_TABLE':'TABLE'},
              'PRT_IND_P': {'PI_P_TABLE':'TABLE'},
              'SEC_IND_P': {'SE_P_TABLE':'TABLE'},
              'PROT_VIG_EPID_P': {'PROT_VIG_EPID_P_TABLE':'TABLE'},
              'EMP_BSE_TEC_P': {'EBT_P_TABLE':'TABLE'},
              'EMP_CRE_CUL_P': {'ICC_P_TABLE':'TABLE'},
              'INN_GES_EMP_P': {'IG_P_TABLE':'TABLE'},
              'INN_PROC_P': {'IPP_P_TABLE':'TABLE'},
              'REG_NORM_REGL_LEG_P': {'RNR_P_TABLE':'TABLE'},
              'CONP_TEC_P': {'CONP_TEC_P_TABLE':'TABLE'},
              'REG_AAD_P': {'AAAD_P_TABLE':'TABLE'},
              'SIG_DIS_P': {'SD_P_TABLE':'TABLE'}
              },
    'ASC_P': {'GEN_CONT_IMP_P': {'GC_I_P_TABLE_5':'TABLE'},
              'PASC_P': {'PASC_FOR_P_TABLE':'TABLE',
               'PASC_TRA_P_TABLE':'TABLE',
               'PASC_GEN_P_TABLE':'TABLE',
               'PASC_CAD_P_TABLE':'TABLE'},
              'DC_P': {'DC_CD_P_TABLE':'TABLE',
               'DC_CON_P_TABLE':'TABLE',
               'DC_TRA_P_TABLE':'TABLE',
               'DC_TRA_P_TABLE':'TABLE'}},
    'FRH_P': {'TES_DOC_P': {'TD_P_TABLE':'TABLE'},
              'TES_MAST_P': {'TM_P_TABLE':'TABLE'},
              'TES_PREG_P': {'TP_P_TABLE':'TABLE'},
              'PROY_INV_DES_P': {'PID_P_TABLE':'TABLE'},
              'PROY_INV_CRE_P': {'INV_CRE_P_TABLE':'TABLE'},
              'PROY_INV_DES_INN_P': {'PF_P_TABLE':'TABLE'},
              'PROY_INV_RESP_SOC_P': {'PE_P_TABLE':'TABLE'},
              'ASE_PRG_ACA_P': {'APGA_P_TABLE':'TABLE'},
              'ASE_CRE_CUR_P': {'ACC_P_TABLE':'TABLE'},
              'ASE_PRG_ONDAS_P': {'APO_P_TABLE':'TABLE'}}
}'''